In [13]:
import jieba
import jieba.posseg as pseg
import os
import re
from gensim import corpora, models, similarities

In [14]:
stopwords = []
i=0
for line in open("data\\train\\vocab.txt.20k", encoding='utf-8'):
    fields = line.split(' ')
    label = fields[0]
    stopwords.append(label)
    i=i+1
    if i==150:
        break

In [15]:
with open("data\\stop.txt", encoding='utf-8') as f:
    for line in f.readlines():
        stopwords.append(line.strip('\n'))
stopwords.append(' ')
stopwords.append('\n')

In [16]:
print(len(stopwords))

2045


In [17]:
def tokenization(filename):     
    result = []    
    with open(filename, 'r',encoding='utf-8') as f: 
        text = f.read()
        words = jieba.cut(text)    
    for word in words:
        if word not in stopwords:
            result.append(word)
    # print(filename, len(result))
    return result

In [18]:
def tokenization(filename):     
    result = []
    for line in open(filename, encoding='utf-8'):
        if len(line.strip())==0:
            continue
        line_word=[]
        words = jieba.cut(line)   
        for word in words:
            if word not in stopwords:
                line_word.append(word)
        result.append(line_word)
    print(filename,len(result))
    return result

In [19]:
corpus = []
dirname = 'data/law'
filenames = []
for root,dirs,files in os.walk(dirname):    
    for f in files:
        # print(os.path.join(root, f))
        # corpus.append(tokenization(os.path.join(root, f))) 
        corpus = corpus+tokenization(os.path.join(root, f))
        filenames.append(os.path.join(root, f))
dictionary = corpora.Dictionary(corpus)
print (len(filenames))


data/law\刑事诉讼法\第一章  刑事诉讼法概述.txt 78
data/law\刑事诉讼法\第七章  刑事证据.txt 211
data/law\刑事诉讼法\第三章  刑事诉讼中的专门机关和诉讼参与人.txt 142
data/law\刑事诉讼法\第九章  附带民事诉讼.txt 51
data/law\刑事诉讼法\第二十一章 当事人和解的公诉案件诉讼程序.txt 39
data/law\刑事诉讼法\第二十三章 依法不负刑事责任的精神病人的强制医疗程序.txt 42
data/law\刑事诉讼法\第二十二章 犯罪嫌疑人、被告人逃匿、死亡案件违法所得的没收程序.txt 48
data/law\刑事诉讼法\第二十四章 涉外刑事诉讼程序与司法协助制度.txt 46
data/law\刑事诉讼法\第二十章  未成年人刑事案件诉讼程序.txt 95
data/law\刑事诉讼法\第二章  刑事诉讼法的基本原则.txt 81
data/law\刑事诉讼法\第五章  回避.txt 40
data/law\刑事诉讼法\第八章  强制措施.txt 180
data/law\刑事诉讼法\第六章  辩护与代理.txt 103
data/law\刑事诉讼法\第十一章  立案.txt 67
data/law\刑事诉讼法\第十七章  死刑复核程序.txt 67
data/law\刑事诉讼法\第十三章  起诉.txt 148
data/law\刑事诉讼法\第十九章  执行.txt 167
data/law\刑事诉讼法\第十二章  侦查.txt 219
data/law\刑事诉讼法\第十五章  第一审程序.txt 416
data/law\刑事诉讼法\第十八章 审判监督程序.txt 124
data/law\刑事诉讼法\第十六章  第二审程序.txt 240
data/law\刑事诉讼法\第十四章  刑事审判概述.txt 184
data/law\刑事诉讼法\第十章  期间、送达.txt 47
data/law\刑事诉讼法\第四章  管辖.txt 72
data/law\刑法\第一章  刑法概说.txt 57
data/law\刑法\第七章  单位犯罪.txt 25
data/law\刑法\第三十一章  妨害社会管理秩序罪（5）：危害公共卫生罪.txt 32
data/law\刑法\第三十七

In [20]:
print(len(corpus))

14978


In [21]:
doc_vectors = [dictionary.doc2bow(text) for text in corpus] 
tfidf = models.TfidfModel(doc_vectors)

In [22]:
import pickle
f = open('data/example/train.pkl', 'rb')
train_example=pickle.load(f)

test_string=train_example[1].context_text
print(test_string)
test_doc_list=[]
words = jieba.cut(test_string)  
print(words)  
for word in words:
    if word not in stopwords:
        test_doc_list.append(word)

test_doc_vec=dictionary.doc2bow(test_doc_list)
index = similarities.SparseMatrixSimilarity(tfidf[doc_vectors], num_features=len(dictionary.keys()))
sim = index[tfidf[test_doc_vec]]
scores=sorted(enumerate(sim), key=lambda item: -item[1])  # 排序
for i in range(3):
    print(corpus[scores[i][0]],scores[i][1])

本院经审理认定事实如下:2014年8月6日,被告与光大银行签订《阳光个人小额信用贷款合同》,约定:被告向x7银贵阳分行行贷款50000元;贷款期限为36个月,自2014年8月6日至2017年8月6日至;贷款年利率按年利率8.61%执行;贷款按月分36期对日偿还(贷款发放日作为每期还款日,还款日为每月6日);若贷款逾期归还,光大银行贵阳分行有权宣布贷款立即到期,并要求借款人立即偿还全部贷款本金、利息即其他应付款项;该合同一式三份,由借款人、贷款人、保险公司各执一份(合同贷款细节约定第12项)随即被告作为投保人,以光大银行贵阳分行为被保险人在原告处投保个人贷款保险,原告向被告出具《阳光个人贷款保证保险单》(保险单号码:P220920140806006956),被告在保险单投保人处签名、捺手印保险单载明:保险金额为59450元;保险期间为贷款发生之日至约定的清偿全部贷款本息之日;每月保费为900元,保费按月支付;投保人拖欠任何一期应偿贷款达到80日,保险人依据保险合同对保险人进行赔偿;保险人向被保险人赔付后,有权向投保人追偿,投保人需向保险人归还全部赔偿款及应付而未付的保险费(应付而未付保险费至投保人自贷款发放之日至理赔之日止未支付的保险费);从保险人赔偿当日开始30天,投保人未向保险人归还全部赔偿款,视为投保人违约,投保人需以尚欠全部款项为基数,从保险人赔偿当日开始计算,按每日万分之五向保险人交纳违约金;保险人有权向投保人催回应付而未付的保险费、赔偿款项、违约金、理赔及催收产生的其他费用同日,光大银行向被告发放贷款50000元,被告在光大银行贵阳分行贷款借据中借款人处签字予以认可2016年2月26日,被告最后一次归还借款,未再履行还款及支付保费义务2016年4月27日,原告按照保单约定,履行代偿义务,分四笔向x7银行贵阳分行代偿本息共计27852.59元光大银行贵阳分行向原告向出具《代偿债务与权益转让确认书》,载明,个人贷款保证保险(保险单号码:P220920140806006956)保单项下的代偿款27852.59元已于2016年4月27日收到,同意将对借款人(投保人)追偿的全部利益转让给原告原告为本案支付了6000元律师费
<generator object Tokenizer.cut at 0x0000018E6E9EDC10>
['当事人', '权利', '义务

In [23]:
for i in range(4):
    print(corpus[scores[i][0]],scores[i][1])

['当事人', '权利', '义务', '借款人', '义务', '归还', '本金', '利息', '无偿', '借款人', '须', '归还', '本金', '有偿', '借款人', '除须', '归还', '本金', '外', '利息'] 0.2501116
['保险合同', '分论', '人身保险', '人身保险', '若干', '规则', '人身保险', '如实', '申报', '被保险人', '年龄', '申报', '被保险人', '年龄', '真实', '将会', '导致', '法律', '后果', '表现', '保险费', '更正', '补交', '保险金', '减少', '申报', '被保险人', '年龄', '真实', '致使', '保险费', '少于', '应付', '保险费', '有权', '更正', '补交', '保险费', '给付', '保险金', '实付', '保险费', '应付', '保险费', '比例'] 0.18264088
['债', '担保', '保证', '保证', '概念', '种类', '保证', '条件', '指出', '担保法', '保证', '资格', '强制性', '保证', '代偿', '能力', '保证', '法律效力', '最高人民法院', '担保法', '解释', '14', '条', '代偿', '能力', '法人', '组织', '自然人', '保证', '身份', '订立', '保证', '代偿', '能力', '免除', '保证', '人民法院', '不予', '支持'] 0.18209678
['保险合同', '总论', '保险合同', '履行', '义务', '保险法', '14', '条', '保险合同', '成立', '交付', '保险费', '时间', '承担', '保险费', '交付', '保险合同', '效力', '先决条件', '保险合同', '期限内', '标的', '危险', '增加', '增加', '保险费', '交纳', '增加', '保险费', '危险', '降低', '减交', '保险费', '被保险人', '履行', '标的', '防损', '增加', '保险费', '增加', '交纳', '保险费', '申报', '被保险人', '年龄', '不实', '致使', 